In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random

In [8]:
class MyLogReg:
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None, reg=None, l1_coef=0, l2_coef=0, sgd_sample=None, random_state=42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = None
        self.metric = metric
        self.best_score = None 
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        self.random_state = random_state
        
    def __str__(self):
        params = vars(self) # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyLogReg class: {params_str}"
    
    # Функция ошибки (Log Loss)
    def log_loss(self, y_true, y_pred):
        epsilon = 1e-15  # Маленькое число для предотвращения деления на 0
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Ограничиваем предсказание
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    # сигмоидная функция активации для вероятностей от 0 до 1
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    #Добавление регуляризации к функции потерь.
    def regularization_loss(self, loss):
 
        if self.reg == 'l1':
            regularization = self.l1_coef * np.sum(np.abs(self.weights[1:]))
        elif self.reg == 'l2':
            regularization = self.l2_coef * np.sum(self.weights[1:] ** 2)
        elif self.reg == 'elasticnet':
            l1_reg = self.l1_coef * np.sum(np.abs(self.weights[1:]))
            l2_reg = self.l2_coef * np.sum(self.weights[1:] ** 2)
            regularization = l1_reg + l2_reg
        else:
            regularization = 0
        return loss + regularization


    #Добавление регуляризации к градиенту.
    def regularization_gradient(self, gradients):
        if self.reg == 'l1':
            gradients[1:] += self.l1_coef * np.sign(self.weights[1:])  # L1 градиент
        elif self.reg == 'l2':
            gradients[1:] += 2 * self.l2_coef * self.weights[1:]  # L2 градиент
        elif self.reg == 'elasticnet':
            gradients[1:] += self.l1_coef * np.sign(self.weights[1:])  # L1 часть
            gradients[1:] += 2 * self.l2_coef * self.weights[1:]  # L2 часть
        return gradients
    
     # Метод обучения модели
    def fit(self, X, y, verbose=False):
        random.seed(self.random_state)
         
        # Добавляем единичный столбец для смещения
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # Инициализация весов: количество фич + 1 (для смещения)
        self.weights = np.ones(X.shape[1])
        
        # Для начального лога ошибки
        y_pred_initial = self.sigmoid(np.dot(X, self.weights))
        loss_initial = self.log_loss(y, y_pred_initial)
        loss_initial = self.regularization_loss(loss_initial)
        
        # Вывод начальной метрики, если она задана
        metric_value_initial = None
        if self.metric:
            metric_value_initial = self._calculate_metric(y, (y_pred_initial >= 0.5).astype(int), y_pred_initial)
            print(f"Start loss: {loss_initial:.2f} i {self.metric}: {metric_value_initial:.2f}")
        else:
            print(f"Start loss: {loss_initial:.2f}")
        
        # Градиентный спуск
        for i in range(self.n_iter):
            
            # Если sgd_sample задан, выбираем подвыборку
            if self.sgd_sample:
                if isinstance(self.sgd_sample, int):
                    sample_size = self.sgd_sample
                elif isinstance(self.sgd_sample, float):
                    sample_size = int(self.sgd_sample * X.shape[0])
                sample_rows_idx = random.sample(range(X.shape[0]), sample_size)
                X_sample = X[sample_rows_idx]
                y_sample = y[sample_rows_idx]
            else:
                X_sample = X
                y_sample = y
                
            # 1. Предсказание (sigmoid)
            y_pred = self.sigmoid(np.dot(X_sample, self.weights))
            
            # 2. Вычисление градиента
            gradient = np.dot(X_sample.T, (y_pred - y_sample)) / y_sample.size
            
            # 3. Добавляем регуляризацию к градиенту
            gradient = self.regularization_gradient(gradient)
            
            # 4. Обновление весов
            if callable(self.learning_rate):  # Если learning_rate — это функция
                current_learning_rate = self.learning_rate(i)
            else:  # Если learning_rate — это число
                current_learning_rate = self.learning_rate
            self.weights -= current_learning_rate * gradient
            
            # 5. Вычисляем функцию потерь на каждой итерации (вне verbose)
            loss = self.log_loss(y, y_pred)
            loss = self.regularization_loss(loss)  # Добавляем регуляризацию к лоссу
            
            # 6. Логирование каждые verbose итераций
            if verbose and (i + 1) % verbose == 0:
                if self.metric:
                    metric_value = self._calculate_metric(y, (y_pred >= 0.5).astype(int), y_pred)
                    print(f"{i + 1} loss: {loss:.2f}, {self.metric}: {metric_value:.2f}")
                else:
                    print(f"{i + 1} loss: {loss:.2f}")
                
             # После обучения сохраняем значение метрики для обученной модели
            if self.metric:
                self.best_score = self._calculate_metric(y, (y_pred >= 0.5).astype(int), y_pred)
    
    # Метод для получения коэффициентов (весов)
    def get_coef(self):
        if self.weights is not None:
            return self.weights
        else:
            raise ValueError("Model is not fitted yet. Please call the fit method first.")
        
     # Метод для предсказания вероятностей классов (логиты через сигмоиду)
    def predict_proba(self, X):
        # Добавляем единичный столбец для смещения
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        # Возвращаем вероятности через сигмоид
        return self.sigmoid(np.dot(X, self.weights))
    
    # Метод для предсказания классов
    def predict(self, X):
        # Получаем вероятности
        probabilities = self.predict_proba(X)
        # Превращаем их в бинарные классы на основе порога 0.5
        return (probabilities >= 0.5).astype(int)
    
     # Реализация Accuracy
    def accuracy(self, y_true, y_pred):
        return np.mean(y_true == y_pred)

    # Реализация Precision
    def precision(self, y_true, y_pred):
        true_positive = np.sum((y_true == 1) & (y_pred == 1))
        false_positive = np.sum((y_true == 0) & (y_pred == 1))
        if true_positive + false_positive == 0:
            return 0.0
        return true_positive / (true_positive + false_positive)

    # Реализация Recall
    def recall(self, y_true, y_pred):
        true_positive = np.sum((y_true == 1) & (y_pred == 1))
        false_negative = np.sum((y_true == 1) & (y_pred == 0))
        if true_positive + false_negative == 0:
            return 0.0
        return true_positive / (true_positive + false_negative)

    # Реализация F1-score
    def f1(self, y_true, y_pred):
        prec = self.precision(y_true, y_pred)
        rec = self.recall(y_true, y_pred)
        if prec + rec == 0:
            return 0.0
        return 2 * (prec * rec) / (prec + rec)

    # Реализация ROC AUC
    def roc_auc(self, y_true, y_proba):
        # Сортируем по вероятностям
        sorted_indices = np.argsort(y_proba)
        y_true_sorted = y_true[sorted_indices]
        y_proba_sorted = y_proba[sorted_indices]
        
        # Вычисление метрики ROC AUC вручную
        tpr = []  # True positive rate
        fpr = []  # False positive rate
        
        P = np.sum(y_true_sorted == 1)  # количество положительных примеров
        N = np.sum(y_true_sorted == 0)  # количество отрицательных примеров
        
        tp = 0  # True positives
        fp = 0  # False positives
        
        for i in range(len(y_true_sorted)):
            if y_true_sorted[i] == 1:
                tp += 1
            else:
                fp += 1
            tpr.append(tp / P)
            fpr.append(fp / N)
        
        # Вычисление площади под кривой (метод трапеций)
        auc = np.trapz(tpr, fpr)
        return auc
    
    # Метод для расчёта метрики
    def _calculate_metric(self, y_true, y_pred, y_proba):
        if self.metric == 'accuracy':
            return self.accuracy(y_true, y_pred)
        elif self.metric == 'precision':
            return self.precision(y_true, y_pred)
        elif self.metric == 'recall':
            return self.recall(y_true, y_pred)
        elif self.metric == 'f1':
            return self.f1(y_true, y_pred)
        elif self.metric == 'roc_auc':
            return self.roc_auc(y_true, y_proba)
        else:
            raise ValueError(f"Unknown metric: {self.metric}")
    
    # Метод для возврата значения метрики после обучения
    def get_best_score(self):
        if self.best_score is not None:
            return self.best_score
        else:
            raise ValueError("No metric was set or model wasn't trained.")

In [20]:
# Генерация синтетических данных
X, y = make_classification(n_samples=100, n_features=5, n_classes=2, random_state=42)

# Определение параметров для тестирования в диапазоне от 0.0 до 1.0
sgd_samples = [0.01]  # Дробные значения представляют долю выборки

# Цикл для тестирования с различными параметрами sgd_sample
for sgd_sample in sgd_samples:
    print(f"\nTesting with sgd_sample = {sgd_sample}")
    
    # Инициализация модели с текущим значением sgd_sample
    model = MyLogReg(n_iter=1000, learning_rate=0.1, sgd_sample=sgd_sample, random_state=42)
    
    # Обучение модели
    model.fit(X, y, verbose=False)
    
    # Получение средних значений коэффициентов
    coef_mean = np.mean(model.get_coef())
    print(f"Среднее значение коэффициентов: {coef_mean:.4f}")


Testing with sgd_sample = 0.01
Start loss: 1.07
Среднее значение коэффициентов: 0.4170


In [30]:
#тестирование динамического спуска

X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение функции динамического изменения learning_rate
dynamic_lr = lambda iter: 0.5 * (0.85 ** iter)

# Создание модели с динамическим шагом обучения
model_dynamic_lr = MyLogReg(n_iter=1000, learning_rate=dynamic_lr, metric='accuracy')

# Обучение модели
model_dynamic_lr.fit(X_train, y_train, verbose=100)

# Предсказания и вывод точности
y_pred_dynamic_lr = model_dynamic_lr.predict(X_test)
accuracy_dynamic_lr = model_dynamic_lr.get_best_score()
print(f"Accuracy with dynamic learning rate: {accuracy_dynamic_lr:.4f}")


Start loss: 2.47 i accuracy: 0.54
100 loss: 0.58, accuracy: 0.73
200 loss: 0.58, accuracy: 0.73
300 loss: 0.58, accuracy: 0.73
400 loss: 0.58, accuracy: 0.73
500 loss: 0.58, accuracy: 0.73
600 loss: 0.58, accuracy: 0.73
700 loss: 0.58, accuracy: 0.73
800 loss: 0.58, accuracy: 0.73
900 loss: 0.58, accuracy: 0.73
1000 loss: 0.58, accuracy: 0.73
Accuracy with dynamic learning rate: 0.7325


In [31]:
#тестирование для fit
X = pd.DataFrame({
    'feature1': [0.2, 0.3, 0.5, 0.7],
    'feature2': [0.5, 0.6, 0.8, 0.9]
})
y = pd.Series([0, 0, 1, 1])

# Инициализируем и обучим модель
model = MyLogReg(n_iter=300, learning_rate=0.1)
model.fit(X, y, verbose=100)

# Получим веса
weights = model.get_coef()
np.mean(weights)


Start loss: 1.02
100 loss: 0.58
200 loss: 0.52
300 loss: 0.47


0.5624206603443257

In [32]:
# тестировка для predict
model = MyLogReg(n_iter=1000, learning_rate=0.1)

# Генерация данных
def test_model(n_samples, n_informative):
    X, y = make_classification(n_samples=n_samples, n_features=10, n_informative=n_informative, 
                               n_redundant=0, random_state=42)
    
    # Преобразуем данные в pandas DataFrame для удобства
    X_df = pd.DataFrame(X)
    y_series = pd.Series(y)
    
    # Обучаем модель
    model.fit(X_df, y_series, verbose=False)
    
    # Получаем предсказания
    predictions = model.predict(X_df)
    probabilities = model.predict_proba(X_df)
    
    # Выводим результаты
    print(f"Сумма предсказанных классов: {np.sum(predictions)}")
    print(f"Среднее вероятностей: {np.mean(probabilities):.6f}")

# Пример 1: n_samples=400, n_informative=5
print("Dataset 1: n_samples=400, n_informative=5")
test_model(n_samples=400, n_informative=5)

# Пример 2: n_samples=500, n_informative=7
print("\nDataset 2: n_samples=500, n_informative=7")
test_model(n_samples=500, n_informative=7)

# Пример 3: n_samples=300, n_informative=3
print("\nDataset 3: n_samples=300, n_informative=3")
test_model(n_samples=300, n_informative=3)

Dataset 1: n_samples=400, n_informative=5
Start loss: 1.82
Сумма предсказанных классов: 197
Среднее вероятностей: 0.502369

Dataset 2: n_samples=500, n_informative=7
Start loss: 2.54
Сумма предсказанных классов: 256
Среднее вероятностей: 0.504064

Dataset 3: n_samples=300, n_informative=3
Start loss: 0.82
Сумма предсказанных классов: 154
Среднее вероятностей: 0.500393


In [33]:
# тестирование метрик
X, y = make_classification(n_samples=400, n_features=10, n_informative=5, random_state=42)

# Преобразуем данные в pandas DataFrame
X_df = pd.DataFrame(X)
y_series = pd.Series(y)

# Инициализируем модель с метрикой f1
model = MyLogReg(n_iter=100, learning_rate=0.1, metric='f1')

# Обучаем модель
model.fit(X_df, y_series, verbose=10)

# Получаем метрику f1 после обучения
best_f1_score = model.get_best_score()
print(f"Лучшее значение f1-score: {best_f1_score:.4f}")

Start loss: 1.40 i f1: 0.62
10 loss: 1.12, f1: 0.62
20 loss: 0.92, f1: 0.65
30 loss: 0.78, f1: 0.67
40 loss: 0.68, f1: 0.67
50 loss: 0.61, f1: 0.69
60 loss: 0.56, f1: 0.72
70 loss: 0.52, f1: 0.74
80 loss: 0.50, f1: 0.76
90 loss: 0.48, f1: 0.78
100 loss: 0.47, f1: 0.79
Лучшее значение f1-score: 0.7879


In [34]:
# тестирование регуляризации
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)

# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Тест 1: Логистическая регрессия без регуляризации
model_no_reg = MyLogReg(n_iter=1000, learning_rate=0.1, metric='accuracy', reg=None)
model_no_reg.fit(X_train, y_train, verbose=100)
y_pred_no_reg = model_no_reg.predict(X_test)
accuracy_no_reg = model_no_reg.get_best_score()
coef_sum_no_reg = np.sum(np.abs(model_no_reg.get_coef()))  # Сумма коэффициентов
print(f"Sum of coefficients without regularization: {coef_sum_no_reg:.4f}")

# Тест 2: Логистическая регрессия с L1 регуляризацией (Lasso)
model_l1_reg = MyLogReg(n_iter=1000, learning_rate=0.1, metric='accuracy', reg='l1', l1_coef=0.01)
model_l1_reg.fit(X_train, y_train, verbose=100)
y_pred_l1 = model_l1_reg.predict(X_test)
accuracy_l1 = model_l1_reg.get_best_score()
coef_sum_l1 = np.sum(np.abs(model_l1_reg.get_coef()))  # Сумма коэффициентов
print(f"Sum of coefficients with L1 regularization: {coef_sum_l1:.4f}")

# Тест 3: Логистическая регрессия с L2 регуляризацией (Ridge)
model_l2_reg = MyLogReg(n_iter=1000, learning_rate=0.1, metric='accuracy', reg='l2', l2_coef=0.01)
model_l2_reg.fit(X_train, y_train, verbose=100)
y_pred_l2 = model_l2_reg.predict(X_test)
accuracy_l2 = model_l2_reg.get_best_score()
coef_sum_l2 = np.sum(np.abs(model_l2_reg.get_coef()))  # Сумма коэффициентов
print(f"Sum of coefficients with L2 regularization: {coef_sum_l2:.4f}")

# Тест 4: Логистическая регрессия с ElasticNet регуляризацией
model_elasticnet_reg = MyLogReg(n_iter=1000, learning_rate=0.1, metric='accuracy', reg='elasticnet', l1_coef=0.01, l2_coef=0.01)
model_elasticnet_reg.fit(X_train, y_train, verbose=100)
y_pred_elasticnet = model_elasticnet_reg.predict(X_test)
accuracy_elasticnet = model_elasticnet_reg.get_best_score()
coef_sum_elasticnet = np.sum(np.abs(model_elasticnet_reg.get_coef()))  # Сумма коэффициентов
print(f"Sum of coefficients with ElasticNet regularization: {coef_sum_elasticnet:.4f}")

Start loss: 2.47 i accuracy: 0.54
100 loss: 0.45, accuracy: 0.80
200 loss: 0.43, accuracy: 0.83
300 loss: 0.43, accuracy: 0.83
400 loss: 0.43, accuracy: 0.83
500 loss: 0.43, accuracy: 0.83
600 loss: 0.43, accuracy: 0.83
700 loss: 0.43, accuracy: 0.83
800 loss: 0.43, accuracy: 0.83
900 loss: 0.43, accuracy: 0.83
1000 loss: 0.43, accuracy: 0.83
Sum of coefficients without regularization: 5.2989
Start loss: 2.57 i accuracy: 0.54
100 loss: 0.49, accuracy: 0.80
200 loss: 0.47, accuracy: 0.83
300 loss: 0.47, accuracy: 0.83
400 loss: 0.47, accuracy: 0.83
500 loss: 0.47, accuracy: 0.83
600 loss: 0.47, accuracy: 0.82
700 loss: 0.47, accuracy: 0.83
800 loss: 0.47, accuracy: 0.83
900 loss: 0.47, accuracy: 0.83
1000 loss: 0.47, accuracy: 0.83
Sum of coefficients with L1 regularization: 3.7181
Start loss: 2.57 i accuracy: 0.54
100 loss: 0.48, accuracy: 0.80
200 loss: 0.46, accuracy: 0.83
300 loss: 0.46, accuracy: 0.83
400 loss: 0.46, accuracy: 0.83
500 loss: 0.46, accuracy: 0.83
600 loss: 0.45, acc